# libraries

In [1]:
import pandas as pd

In [2]:
import requests

# data path

In [3]:
data_path = "../data/"

In [4]:
media_link_dat_filename = "media_government.xlsx"

In [5]:
media_link_dat = pd.read_excel(data_path+media_link_dat_filename)
media_link_dat.columns = [label.strip() for label in media_link_dat.columns]
media_link_dat.columns

Index(['Title', 'Source', 'Data', 'Link', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [6]:
select_columns = ["Source","Data","Link"]
media_link_dat = media_link_dat[select_columns]
media_link_dat.head()

,Source,Data,Link
0,Хабар,5.01.2022,https://khabar.kz/ru/news/obshchestvo/item/137...
1,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137784...
2,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137785...
3,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137786...
4,Хабар,5.01.2022,https://khabar.kz/ru/news/obshchestvo/item/137...


In [24]:
media_link_dat.shape

(510, 3)

In [30]:
media_link_dat = media_link_dat.drop_duplicates()
media_link_dat.shape

(495, 3)

In [31]:
list(set(media_link_dat.Source.to_list()))

['Хабар', 'tengrinews.kz', 'nur.kz', 'akorda.kz', 'sputnik.kz', 'zakon.kz']

In [32]:
cluster1_agencies = ['akorda.kz','Хабар','nur.kz']
cluster2_agencies = ['sputnik.kz','zakon.kz','tengrinews.kz']

In [33]:
cluster1_url = media_link_dat.loc[media_link_dat.Source.isin(cluster1_agencies),"Link"].tolist()
cluster1_url

['https://khabar.kz/ru/news/obshchestvo/item/137781-khronologiya-protestov-v-mangistauskoj-oblasti',
 'https://khabar.kz/ru/news/politika/item/137784-rezhim-chp-v-mangistauskoj-oblasti-i-almaty-prodlitsya-do-19-yanvarya',
 'https://khabar.kz/ru/news/politika/item/137785-prezident-kazakhstana-nazval-vinovnykh-v-dopushchenii-protestnoj-situatsii',
 'https://khabar.kz/ru/news/politika/item/137786-k-tokaev-odna-iz-glavnejshikh-zadach-ispolnitelnoj-vlasti-ostalas-nevypolnennoj',
 'https://khabar.kz/ru/news/obshchestvo/item/137792-kakoj-ushcherb-nanesli-uchastniki-mitingov',
 'https://khabar.kz/ru/news/obshchestvo/item/137801-190-postradavshikh-v-nochnykh-besporyadkakh-obratilis-za-medpomoshchyu-v-almaty',
 'https://khabar.kz/ru/news/politika/item/137805-o-vvedenii-chrezvychajnogo-polozheniya-v-gorode-nur-sultane',
 'https://khabar.kz/ru/news/obshchestvo/item/137818-khronologiya-protestov-v-kazakhstane',
 'https://khabar.kz/ru/news/politika/item/137808-prezident-kazakhstana-provel-zasedanie-

In [34]:
len(cluster1_url)

189

In [35]:
cluster2_url = media_link_dat.loc[media_link_dat.Source.isin(cluster2_agencies),"Link"].tolist()
cluster2_url

['https://www.zakon.kz/6003471-deputaty-mazhilisa-vstretilis-s-protestuiushchimi-zhiteliami-aktau.html',
 'https://www.zakon.kz/6003530-protestuiushchie-v-almaty-razoshlis.html',
 'https://www.zakon.kz/6003583-mitinguiushchie-shturmuiut-akimat-almaty.html',
 'https://www.zakon.kz/6003593-komendant-almaty-prizval-mitinguiushchikh-razoitis-i-izbegat-nasiliia.html',
 'https://www.zakon.kz/6003545-osobaia-vina-za-dopushchenie-protestnoi-situatsii-lezhit-na-pravitelstve-tokaev.html',
 'https://www.zakon.kz/6003569-nurlan-nogaev-prizval-protestuiushchikh-razoitis.html',
 'https://www.zakon.kz/6003580-zarubezhnye-smi-podkliuchilis-k-osveshcheniiu-protestov-v-kazakhstane.html',
 'https://www.zakon.kz/6003713-90-tysiach-tenge-predlozhili-inostrantsu-za-uchastie-v-mitinge-v-almaty.html',
 'https://www.zakon.kz/6003749-okolo-500-zhitelei-pavlodarskoi-oblasti-nakazany-posle-mitingov-u-zdaniia-akimata.html',
 'https://www.zakon.kz/6003766-veteran-spetssluzhb-u-mitinguiushchikh-est-trebovaniia-u-ter

In [36]:
len(cluster2_url)

306

In [37]:
media_link_dat.shape

(495, 3)

In [38]:
len(cluster1_url) + len(cluster2_url)

495

# analysis path

In [39]:
analysis_path = "../analysis/"

In [40]:
keywords1_filename = "keywords_cluster1.xlsx"
keywords2_filename = "keywords_cluster2.xlsx"

In [41]:
keywords1_df = pd.read_excel(analysis_path+keywords1_filename)
keywords2_df = pd.read_excel(analysis_path+keywords2_filename)

In [42]:
keywords1 = keywords1_df.Russian.tolist()
keywords1

['Государство',
 'Закон',
 'Депутат',
 'Народ',
 'Интерес',
 'Реформа',
 'Инициатива',
 'Население']

In [43]:
keywords2 = keywords2_df.Russian.tolist()
keywords2

['Митинг', 'Толпа', 'Беспорядок', 'Военный']

# step 0. define function to analyze articles by cluster

In [44]:
# scan articles for Cluster 1 & Cluster 2 keywords

def analyze_keywords_by_cluster(cluster,url_cluster,keywords_cluster1,keywords_cluster2):
    
    cluster_df = pd.DataFrame(data={"cluster":[],"keyword1_count":[],"keyword2_count":[],"url":[]})
    
    print(cluster_df)

    for url in url_cluster:
        response = requests.get(url)

        # analyze Cluster 1 keywords
        kcount1_total = 0
        for k in keywords_cluster1:
            kcount = response.text.count(k)
            kcount1_total += kcount
            # print(k,kcount)
            
        # analyze Cluster 2 keywords
        kcount2_total = 0
        for k in keywords_cluster2:
            kcount = response.text.count(k)
            kcount2_total += kcount
            # print(k,kcount)

        print(response,cluster,kcount1_total,kcount2_total)

        # Row(s) to append
        new_rows = pd.DataFrame({'cluster':[cluster],'keyword1_count':[kcount1_total],'keyword2_count':[kcount2_total],'url':[url]})
        # print(new_rows)
        
        cluster_df = pd.concat([cluster_df,new_rows])
        
    return cluster_df

# step 1. analyze Cluster 1 articles

In [45]:
cluster1_url[0:5]

['https://khabar.kz/ru/news/obshchestvo/item/137781-khronologiya-protestov-v-mangistauskoj-oblasti',
 'https://khabar.kz/ru/news/politika/item/137784-rezhim-chp-v-mangistauskoj-oblasti-i-almaty-prodlitsya-do-19-yanvarya',
 'https://khabar.kz/ru/news/politika/item/137785-prezident-kazakhstana-nazval-vinovnykh-v-dopushchenii-protestnoj-situatsii',
 'https://khabar.kz/ru/news/politika/item/137786-k-tokaev-odna-iz-glavnejshikh-zadach-ispolnitelnoj-vlasti-ostalas-nevypolnennoj',
 'https://khabar.kz/ru/news/obshchestvo/item/137792-kakoj-ushcherb-nanesli-uchastniki-mitingov']

In [46]:
# analyze Cluster 1 articles
cluster1_df = analyze_keywords_by_cluster(1,cluster1_url,keywords1,keywords2)

Empty DataFrame
Columns: [cluster, keyword1_count, keyword2_count, url]
Index: []
<Response [200]> 1 2 2
<Response [200]> 1 6 0
<Response [200]> 1 2 0
<Response [200]> 1 4 4
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 11 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 10 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 2 0
<Response [200]> 1 

ReadTimeout: HTTPSConnectionPool(host='khabar.kz', port=443): Read timed out. (read timeout=None)

In [23]:
cluster1_df

NameError: name 'cluster1_df' is not defined

# step 2. analyze Cluster 2 articles

In [ ]:
# analyze Cluster 2 articles
cluster2_df = analyze_keywords_by_cluster(2,cluster2_url,keywords1,keywords2)

In [ ]:
cluster2_df

# export files

In [ ]:
cluster_df = pd.concat([cluster1_df,cluster2_df])
cluster_df.shape

In [ ]:
# article_kcounts_cluster1_filename = "article_kcounts_cluster1.xlsx"
# cluster1_df.to_excel(analysis_path+article_kcounts_cluster1_filename,index=False)

In [ ]:
# article_kcounts_cluster2_filename = "article_kcounts_cluster2.xlsx"
# cluster2_df.to_excel(analysis_path+article_kcounts_cluster2_filename,index=False)

In [ ]:
article_kcounts_cluster_filename = "article_kcounts_cluster.xlsx"
cluster_df.to_excel(analysis_path+article_kcounts_cluster_filename,index=False)

# import file, remove duplicates, and export again

In [49]:
exported_filename = "article_kcounts_merged.xlsx"
df = pd.read_excel(analysis_path + exported_filename)

In [50]:
df.shape

(544, 10)

In [52]:
df = df.drop_duplicates()
df.shape

(495, 10)

In [53]:
article_kcounts_cluster_filename = "article_kcounts_cluster.xlsx"
df.to_excel(analysis_path+article_kcounts_cluster_filename,index=False)